<a href="https://colab.research.google.com/github/Venu212/Finding-Donors/blob/master/deepface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep Face

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_util import *
from inception_block import *


In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
! wget https://drive.google.com/open?id=1YAzIBtWbI969m5nfRwqN8SUFuIQcMWZw

--2019-05-16 07:34:00--  https://drive.google.com/open?id=1YAzIBtWbI969m5nfRwqN8SUFuIQcMWZw
Resolving drive.google.com (drive.google.com)... 74.125.142.139, 74.125.142.101, 74.125.142.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.139|:443... connected.
HTTP request sent, awaiting response... 307 OK
Location: https://drive.google.com/drive/folders/1YAzIBtWbI969m5nfRwqN8SUFuIQcMWZw?usp=drive_open [following]
--2019-05-16 07:34:00--  https://drive.google.com/drive/folders/1YAzIBtWbI969m5nfRwqN8SUFuIQcMWZw?usp=drive_open
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘open?id=1YAzIBtWbI969m5nfRwqN8SUFuIQcMWZw.1’

open?id=1YAzIBtWbI9     [   <=>              ] 222.62K   540KB/s    in 0.4s    

2019-05-16 07:34:01 (540 KB/s) - ‘open?id=1YAzIBtWbI969m5nfRwqN8SUFuIQcMWZw.1’ saved [227958]



Create the model for face images.

In [0]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [43]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [0]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum((anchor - positive) ** 2, axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum((anchor - negative) ** 2, axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = pos_dist - neg_dist + alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))
    return loss
  

In [45]:


with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))



loss = 528.1427


Loading the trained model

In [0]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [0]:
Loading the trained model

Applying the model

Face Verification

In [55]:
database = {}
# database["skuli"] = img_to_encoding("gdrive/My Drive/Colab Notebooks/images/skuli.jpg", FRmodel)
## database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
##database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("gdrive/My Drive/Colab Notebooks/images/andrew.jpg", FRmodel)
"""
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel) 
"""

'\ndatabase["kian"] = img_to_encoding("images/kian.jpg", FRmodel)\ndatabase["dan"] = img_to_encoding("images/dan.jpg", FRmodel)\ndatabase["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)\ndatabase["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)\ndatabase["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)\ndatabase["felix"] = img_to_encoding("images/felix.jpg", FRmodel)\ndatabase["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)\ndatabase["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel) \n'

In [0]:
# GRADED FUNCTION: verify

def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be a resident of the Happy house.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
    ### START CODE HERE ###
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(database[identity] - encoding)
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    ### END CODE HERE ###
        
    return dist, door_open

In [0]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

In [0]:


verify("images/camera_0.jpg", "younes", database, FRmodel)



 Face Recognition

In [0]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = np.linalg.norm(db_enc - encoding)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist < min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity



In [60]:
  

who_is_it("images/camera_0.jpg", database, FRmodel)



TypeError: ignored